In [ ]:
import os
import json
import numpy as np
import pandas as pd
import jinja2 as jj
import urllib.request as ur
from ipyaggrid import Grid


In [ ]:
url = 'https://raw.githubusercontent.com/ag-grid/ag-grid-docs/master/src/olympicWinners.json'
with ur.urlopen(url) as res:
    data = json.loads(res.read().decode('utf-8'))

columnDefs = [
    {'headerName': "Country", 'field': "country", 'rowGroup':True, 'hide':True},
    {'headerName': "Sport", 'field': "sport", 'rowGroup':True, 'hide':True},
    {
        'headerName': "Results",
        'children': [
            {'headerName': "Total", 'field': 'total', 'columnGroupShow': 'closed', 'aggFunc':'sum'},
            {'headerName': "Gold", 'field': 'gold', 'columnGroupShow': 'open', 'aggFunc':'sum'},
            {'headerName': "Silver", 'field': 'silver', 'columnGroupShow': 'open', 'aggFunc':'sum'},
            {'headerName': "Bronze", 'field': 'bronze', 'columnGroupShow': 'open', 'aggFunc':'sum'}
        ]
    },
]


grid_options = {
    'columnDefs': columnDefs,
    'enableFilter': True,
    'enableSorting':True,
}

g = Grid(grid_data = data,
            grid_options=grid_options, 
            export_mode="buttons",
            theme='ag-theme-balham')
g

In [ ]:
df = g.grid_data_out['grid']
df.head()

## Save df html
Must:
+ use notebook.css
+ wrap dataframe.html in specific classes - like in notebook
Result can be iframed in any doc


In [ ]:
%%writefile templates/index.tpl.html

<!DOCTYPE html>
<html>

<head>
    <meta charset="utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <title>dataframe</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" href="https://cdn.jupyter.org/notebook/5.6.0/style/style.min.css">
</head>

<body>


    <div class="output_are">
        <div class="output_subarea output_html rendered_html output_result">

            __$data.df_html$__

        </div>
    </div>

</body>

</html>

In [ ]:
dir_template = 'templates'
dir_dump = 'dump'

loader = jj.FileSystemLoader(dir_template)
env = jj.Environment(loader=loader,
                     variable_start_string='__$',
                     variable_end_string='$__',
                     block_start_string='{-%',
                     block_end_string='%-}'
                     )

template = env.get_template('index.tpl.html')
data = {'df_html': df.head(10).to_html()}
content = template.render(data=data)

if not os.path.exists(dir_dump):
    os.makedirs(dir_dump)

path = os.path.join(dir_dump, 'index.html')
with open(path, 'w') as f:
    f.write(content)
    print('file {} saved to disk'.format(path))


In [ ]:
!cd dump && python -m http.server 8080